In [122]:
import pandas as pd
from datetime import timedelta, datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm

In [147]:
interval_time = 1800

# Format into set ready for training and eval
df = pd.read_csv("feat_eng_"+str(interval_time)+".csv")

# Days in the past
days_in_past = 0

# [{ID, [entries], label}]
data = []
X = dict()
Y = dict()

# Store max call and sms for later normalization
max_call = df["call"].max()
max_sms = df["sms"].max()

df["timestamp"] = pd.to_datetime(df["timestamp"])

for id in df["ID"].unique():
    X[id] = dict()
    Y[id] = dict()
    df_id = df[df["ID"] == id]
    for day in df_id["timestamp"].dt.floor("D").unique():
        
        df_id_days = df_id[((df_id["timestamp"].dt.date <= day.date()) & (df_id["timestamp"].dt.date >= (day - timedelta(days=days_in_past)).date()))].sort_values(by="timestamp", ascending=False)
        label = df_id_days["next_mood"].iloc[0]
        df_id_days = df_id_days.sort_values(by="timestamp", ascending=True).drop(columns=["ID"])

        # being an RNN, timestamp shouldn't be needed since the order is the important
        df_id_days["timestamp"] = df_id_days["timestamp"].astype(int) // 10**11
        df_id_days["timestamp"] = df_id_days["timestamp"] - df_id_days["timestamp"].min()

        df_id_days = df_id_days.drop(columns="Unnamed: 0").reset_index(drop=True)

        # Normalize values
        # Normalize mood, mood_day-1, mood_day-2, next_mood [1:10]
        for col in ["mood", "mood_day-1", "mood_day-2", "next_mood"]:
            df_id_days[col] = (df_id_days[col] - 1)/(10-1)
        # Normalize arousal and valence [-2;2]
        df_id_days["arousal"] = (df_id_days["arousal"] - (-2))/(2 - (-2))
        df_id_days["valence"] = (df_id_days["valence"] - (-2))/(2 - (-2))
        # Activity is already normalized
        # Normalize call, sms [0; max_call], [0; max_sms]
        df_id_days["call"] = (df_id_days["call"] - 0)/(max_call - 0)
        df_id_days["sms"] = (df_id_days["sms"] - 0)/(max_sms - 0)
        # Normalize screen, appCat.X
        timeBasedFields = ['screen', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.other','appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities','appCat.weather']
        for col in timeBasedFields:
            df_id_days[col] = (df_id_days[col] - 0)/(interval_time - 0)
            
        # Using only one value since it will be the next day predicted value
        X[id][day] , Y[id][day] = df_id_days.drop(columns=["next_mood", "timestamp"]), df_id_days["next_mood"][0]
        #raw_tuples = list(df_id_days.itertuples(index=False, name=None))
        #print(len(raw_tuples[0]))
        #data.append({"ID": id, "entries": raw_tuples, "label": label})


In [148]:
import random

train_ids = random.sample(list(X.keys()), int(len(X)*0.75))
print("Number of ids for training: "+str(len(train_ids)))
print("Number of ids for testing: "+ str(len(X) - len(train_ids)))

X_train = []
Y_train = []

X_test = []
Y_test = []

for id in X.keys():
    is_train = True if id in train_ids else False
    for day in X[id].keys():
        if is_train:
            X_train.append(X[id][day])
            Y_train.append(Y[id][day])
        else:
            X_test.append(X[id][day])
            Y_test.append(Y[id][day])

X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test = np.array(X_test)
Y_test = np.array(Y_test)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

Number of ids for training: 20
Number of ids for testing: 7
(939, 48, 21)
(939,)
(306, 48, 21)
(306,)


In [149]:
# Convert to pytorch tensors
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

Y_train_tensors = Variable(torch.Tensor(Y_train))
Y_test_tensors = Variable(torch.Tensor(Y_test))

print(X_train_tensors.shape)
print(X_test_tensors.shape) 

print(Y_train_tensors.shape)
print(Y_test_tensors.shape) 

# Reshaping to rows, timestamps, features
X_train_tensors_final = torch.reshape(X_train_tensors,   
                                      (X_train_tensors.shape[0], 48, 
                                       X_train_tensors.shape[2]))
X_test_tensors_final = torch.reshape(X_test_tensors,  
                                     (X_test_tensors.shape[0], 48, 
                                      X_test_tensors.shape[2])) 



# Apparently there are some nan values in training tensors

print(torch.isnan(X_train_tensors_final).any())
print(torch.isnan(Y_train_tensors).any())
print(torch.isnan(X_test_tensors_final).any())
print(torch.isnan(Y_test_tensors).any())



# Removing nan
X_train_tensors_final = torch.where(torch.isnan(X_train_tensors_final), torch.zeros_like(X_train_tensors_final), X_train_tensors_final)

X_test_tensors_final = torch.where(torch.isnan(X_test_tensors_final), torch.zeros_like(X_test_tensors_final), X_test_tensors_final)


print("Training Shape:", X_train_tensors_final.shape, Y_train_tensors.shape)
print("Testing Shape:", X_test_tensors_final.shape, Y_test_tensors.shape) 

torch.Size([939, 48, 21])
torch.Size([306, 48, 21])
torch.Size([939])
torch.Size([306])
tensor(True)
tensor(False)
tensor(True)
tensor(False)
Training Shape: torch.Size([939, 48, 21]) torch.Size([939])
Testing Shape: torch.Size([306, 48, 21]) torch.Size([306])


In [160]:
class LSTM(nn.Module):
    
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_classes = num_classes  # output size
        self.num_layers = num_layers  # number of recurrent layers in the lstm
        self.input_size = input_size  # input size
        self.hidden_size = hidden_size  # neurons in each lstm layer
        # LSTM model
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=0.2 if num_layers > 1 else 0)
        self.fc_1 = nn.Linear(hidden_size, 128)  # fully connected 
        self.fc_2 = nn.Linear(128, num_classes)  # fully connected last layer
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # hidden state
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # cell state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0))  # (input, hidden, and cell state)
        hn = hn[-1]  # last layer's hidden state
        out = self.relu(hn)
        out = self.fc_1(out)  # first dense
        out = self.relu(out)  # relu
        out = self.fc_2(out)  # final output
        return out

########################################################################################################################################################

def training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train, X_test, y_test):
    
    for epoch in range(n_epochs):

        # Training mode
        lstm.train() 
        # Reset gradients
        optimiser.zero_grad()
        # Forward propagation
        outputs = lstm(X_train)
        # Training loss
        loss = loss_fn(outputs, y_train) 
        # Backpropagate
        loss.backward()
        # Update weights
        torch.nn.utils.clip_grad_norm_(lstm.parameters(), max_norm=1)
        optimiser.step() 

        # Evaluation mode
        lstm.eval() 
        # Disable gradient calc
        with torch.no_grad():
            # Compute classes and losses
            test_preds = lstm(X_test)
            test_loss = loss_fn(test_preds, y_test)
        if epoch % 100 == 0:
            print(f"Epoch: {epoch}, train loss: {loss.item():.5f}, test loss: {test_loss.item():.5f}")
            print("Result std: "+str(np.std(np.array((10-1)*y_test+1))))
            print("Prediction std: "+str(np.std(np.array((10-1)*test_preds+1))))

    return test_preds, y_test

########################################################################################################################################################

import warnings
warnings.filterwarnings('ignore')

# Learning rate 0.01, 0.005, 0.001, 0.0005
# Hidden_size 2, 3, 4, 6, 8, 10, 12, 14, 18, 22, 25, 30
# Num layers 1, 2, 3, 4, 5, 6
# Loss functions  L1Loss, MSELoss

n_epochs = 1000 # 1000 epochs
learning_rate = 0.001 # 0.001 lr

input_size = 21 # number of features
hidden_size = 15 # number of features in hidden state
num_layers = 1 # number of stacked lstm layers

num_classes = 1 # number of output classes 

lstm = LSTM(num_classes, 
              input_size, 
              hidden_size, 
              num_layers)

loss_fn = torch.nn.MSELoss()    # mean-squared error for regression
optimiser = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

prediction, result = training_loop(n_epochs=n_epochs,
              lstm=lstm,
              optimiser=optimiser,
              loss_fn=loss_fn,
              X_train=X_train_tensors_final,
              y_train=Y_train_tensors,
              X_test=X_test_tensors_final,
              y_test=Y_test_tensors)

prediction = (10-1)*prediction+1
result = (10-1)*result+1

print("Expected std: "+str(np.std(np.array(result))))
print("Infer std: "+str(np.std(np.array(prediction))))

print("Learning rate: "+str(learning_rate))
print("Hidden Size: "+str(hidden_size))
print("LSTM layers: "+str(num_layers))
print("Loss function: MSE")

print("Number of different test predictions: "+str(len(np.unique(prediction.numpy()))))
print("First 10 values: "+str(prediction.numpy()[:10]))

########################################################################################################################################################

import matplotlib.pyplot as plt

# Calculate absolute differences
differences = [abs(e - m) for e, m in zip(result, prediction)]

# Sort indices based on differences
sorted_indices = sorted(range(len(differences)), key=lambda k: differences[k])

# Reorder lists based on sorted indices
expected_output_sorted = [prediction[i] for i in sorted_indices]
model_results_sorted = [result[i] for i in sorted_indices]
differences_sorted = [differences[i] for i in sorted_indices]

# Plot the points
plt.figure(figsize=(8, 6))
plt.scatter(range(len(expected_output_sorted)), expected_output_sorted, label='Expected Output')
plt.scatter(range(len(model_results_sorted)), model_results_sorted, label='Model Results')
plt.ylabel('Value')
plt.title('Expected Output vs. Model Results (Ordered by Difference)')
plt.legend()
plt.grid(True)
plt.show()

Epoch: 0, train loss: 0.43202, test loss: 0.45549
Result std: 0.7460366
Prediction std: 0.02107735
Epoch: 100, train loss: 0.22586, test loss: 0.24218
Result std: 0.7460366
Prediction std: 0.065615095
Epoch: 200, train loss: 0.00781, test loss: 0.00821
Result std: 0.7460366
Prediction std: 0.2743955
Epoch: 300, train loss: 0.00753, test loss: 0.00810
Result std: 0.7460366
Prediction std: 0.23903398
Epoch: 400, train loss: 0.00733, test loss: 0.00791
Result std: 0.7460366
Prediction std: 0.21309392
Epoch: 500, train loss: 0.00717, test loss: 0.00775
Result std: 0.7460366
Prediction std: 0.18890655
Epoch: 600, train loss: 0.00705, test loss: 0.00763
Result std: 0.7460366
Prediction std: 0.16665898
Epoch: 700, train loss: 0.00695, test loss: 0.00753
Result std: 0.7460366
Prediction std: 0.14657158
Epoch: 800, train loss: 0.00687, test loss: 0.00746
Result std: 0.7460366
Prediction std: 0.12867999
Epoch: 900, train loss: 0.00681, test loss: 0.00740
Result std: 0.7460366
Prediction std: 0.1

KeyboardInterrupt: 

In [ ]:
class MoodLSTMRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(nn.LSTM, self)__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)         # 10 for 10 classes

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, (_, _) = self.lstm(x, (h0.detach(), c0.detach()))

        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

In [ ]:
model = MoodLSTMRegression(input_dim = 19, hidden_dim = 10, num_layers = 1, output_dim = 1)

In [ ]:
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
model.train()
loss_result = []
for epoch in range(100):
    for inputs, labels in train_loader: 
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backwards()
        optimizer.step()
        loss_result.append(loss.item())
        print(f"Epoch {epoch+1}, Loss {loss.item()}")
